In [414]:
using DataFrames, DataFramesMeta
using Printf
using LinearAlgebra
using CSV

List files in the data directory:

In [4]:
readdir("../data")

3-element Array{String,1}:
 "winemag-data-130k-v2.csv"  
 "winemag-data-130k-v2.json" 
 "winemag-data_first150k.csv"

Lets check the last file ("winemag-data_first150k.csv")

In [5]:
df = CSV.read("../data/winemag-data_first150k.csv")
df[1:5,:]

,Column1,country,description,designation,points,price,province,region_1,region_2,variety,winery
,Int64⍰,String⍰,String⍰,String⍰,Int64⍰,Float64⍰,String⍰,String⍰,String⍰,String⍰,String⍰
1,0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
2,1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,missing,Tinta de Toro,Bodega Carmen Rodríguez
3,2,US,"Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g/L of residual sugar.",Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
4,3,US,"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032.",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
5,4,France,"This is the top wine from La Bégude, named after the highest point in the vineyard at 1200 feet. It has structure, density and considerable acidity that is still calming down. With 18 months in wood, the wine has developing an extra richness and concentration. Produced by the Tari family, formerly of Château Giscours in Margaux, it is a wine made for aging. Drink from 2020.",La Brûlade,95,66.0,Provence,Bandol,missing,Provence red blend,Domaine de la Bégude


**Lets see the headers we have available:**

In [6]:
print(names(df))

Symbol[:Column1, :country, :description, :designation, :points, :price, :province, :region_1, :region_2, :variety, :winery]

In [7]:
nrows, ncols = size(df)
for row in 1:nrows
    for col in 7
        if ismissing(df[row,col])
            println("$(names(df)[col]) value for $(row) is missing!")
        end
    end
end

province value for 1134 is missing!
province value for 1441 is missing!
province value for 68227 is missing!
province value for 113017 is missing!
province value for 135697 is missing!


In [8]:
deleterows!(df, [1134, 1441, 68227, 113017, 135697]);

,Column1,country,description,designation,points,price,province,region_1,region_2,variety,winery
,Int64⍰,String⍰,String⍰,String⍰,Int64⍰,Float64⍰,String⍰,String⍰,String⍰,String⍰,String⍰
1,0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
2,1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,missing,Tinta de Toro,Bodega Carmen Rodríguez
3,2,US,"Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g/L of residual sugar.",Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
4,3,US,"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032.",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
5,4,France,"This is the top wine from La Bégude, named after the highest point in the vineyard at 1200 feet. It has structure, density and considerable acidity that is still calming down. With 18 months in wood, the wine has developing an extra richness and concentration. Produced by the Tari family, formerly of Château Giscours in Margaux, it is a wine made for aging. Drink from 2020.",La Brûlade,95,66.0,Provence,Bandol,missing,Provence red blend,Domaine de la Bégude
6,5,Spain,"Deep, dense and pure from the opening bell, this Toro is a winner. Aromas of dark ripe black fruits are cool and moderately oaked. This feels massive on the palate but sensationally balanced. Flavors of blackberry, coffee, mocha and toasty oak finish spicy, smooth and heady. Drink this exemplary Toro through 2023.",Numanthia,95,73.0,Northern Spain,Toro,missing,Tinta de Toro,Numanthia
7,6,Spain,"Slightly gritty black-fruit aromas include a sweet note of pastry along with a hint of prune. Wall-to-wall saturation ensures that all corners of one's mouth are covered. Flavors of blackberry, mocha and chocolate are highly impressive and expressive, while this settles nicely on a long finish. Drink now through 2024.",San Román,95,65.0,Northern Spain,Toro,missing,Tinta de Toro,Maurodos
8,7,Spain,"Lush cedary black-fruit aromas are luxe and offer notes of marzipan and vanilla. This bruiser is massive and tannic on the palate, but still lush and friendly. Chocolate is a key flavor, while baked berry and cassis flavors are hardly wallflowers. On the finish, this is tannic and deep as a sea trench. Drink this saturated black-colored Toro through 2023.",Carodorum Único Crianza,95,110.0,Northern Spain,Toro,missing,Tinta de Toro,Bodega Carmen Rodríguez
9,8,US,"This re-named vineyard was formerly bottled as deLancellotti. You'll find striking minerality underscoring chunky black fruits. Accents of citrus and graphite comingle, with exceptional midpalate concentration. This is a wine to cellar, though it is already quite enjoyable. Drink now through 2030.",Silice,95,65.0,Oregon

**Lets reduce the size of the dataset to wines in Oregon (mostly because I'm running out of time doing this)**

In [252]:
@linq oregon = df |>
where(:province .== "Oregon")

,Column1,country,description,designation,points,price,province,region_1,region_2,variety,winery
,Int64⍰,String⍰,String⍰,String⍰,Int64⍰,Float64⍰,String⍰,String⍰,String⍰,String⍰,String⍰
1,3,US,"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032.",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
2,8,US,"This re-named vineyard was formerly bottled as deLancellotti. You'll find striking minerality underscoring chunky black fruits. Accents of citrus and graphite comingle, with exceptional midpalate concentration. This is a wine to cellar, though it is already quite enjoyable. Drink now through 2030.",Silice,95,65.0,Oregon,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström
3,11,US,"From 18-year-old vines, this supple well-balanced effort blends flavors of mocha, cherry, vanilla and breakfast tea. Superbly integrated and delicious even at this early stage, this wine seems destined for a long and savory cellar life. Drink now through 2028.",Estate Vineyard Wadensvil Block,95,48.0,Oregon,Ribbon Ridge,Willamette Valley,Pinot Noir,Patricia Green Cellars
4,12,US,"A standout even in this terrific lineup of 2015 releases from Patricia Green, the Weber opens with a burst of cola and tobacco scents and accents. It continues, subtle and detailed, with flavors of oranges, vanilla, tea and milk chocolate discreetly threaded through ripe blackberry fruit.",Weber Vineyard,95,48.0,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Patricia Green Cellars
5,14,US,"With its sophisticated mix of mineral, acid and tart fruits, this seductive effort pleases from start to finish. Supple and dense, it's got strawberry, blueberry, plum and black cherry, a touch of chocolate, and that underlying streak of mineral. All these elements are in good proportion and finish with an appealing silky texture. It's delicious already, but give it another decade for full enjoyment. Drink now through 2028.",Grace Vineyard,95,185.0,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Domaine Serene
6,15,US,"First made in 2006, this succulent luscious Chardonnay is all about minerality. It's got a rich core of butterscotch and the seemingly endless layers of subtle flavors that biodynamic farming can bring. It spends 18 months on the lees prior to bottling. Drink now through 2028.",Sigrid,95,90.0,Oregon,Willamette Valley,Willamette Valley,Chardonnay,Bergström
7,26,US,"Bergström has made a Shea designate since 2003, intent on showcasing a “pretty” style of the vineyard. Here are lovely aromatics, with grape jelly, rose petals and plum in the nose, and a complex run of red fruits in the mouth. Streaks of cola, brown sugar and more come up in a generous finish.",Shea Vineyard,94,62.0,Oregon,Willamette Valley,missing,Pinot Noir,Bergström
8,27,US,"Focused and dense, this intense wine captures the essence of ripe raspberries. Aged in 50% new French oak, it's got a strong vein of coffee and mocha. The flavors broaden out and penetrate, finishing with highlights of cedar, cinnamon and dark chocolate. Drink now through the 2020s, but decant if you want the full expression.",Abetina,94,105.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
9,68,US,"Big, bold, dark and chewy, this builds upon substantial black-cherry fruit and supporting acids. Still knitting together, it shows highlights of cedar and tobacco, along with thick tannins. Give this a couple more years of bottle age, and drink after 2018.",Schindler Vineyard,91,50.0,Oregon,Eola-Amity Hills,Willamette Valley,Pinot Noir,Panther Creek


In [415]:
varieties = unique(oregon[:variety])
wineries = unique(oregon[:winery])

n_varieties = length(varieties)
n_wineries = length(wineries)
@printf("The number of wines rated in Oregon: %d\n", size(oregon,1))
@printf("The number of wineries rated in Oregon: %d\n", n_wineries)
@printf("The number of different variety of wines in Oregon: %d\n", n_varieties)

The number of wines rated in Oregon: 4589
The number of wineries rated in Oregon: 371
The number of different variety of wines in Oregon: 67


# Making our data matrix

In [417]:
A = zeros(Float64, n_varieties,n_wineries)
x = 0
for i = 1:n_varieties
    for u = 1:n_wineries
        @linq this_winery = oregon |> where(:winery .== wineries[u])
        @linq this_wineries_wine = this_winery |> where(:variety .== varieties[i])
        n_wines = nrow(this_wineries_wine)
        if n_wines > 0
            A[i, u] = sum(this_wineries_wine[:points])/n_wines
        else
            A[i, u] = -1
        end
    end
end
A

67×371 Array{Float64,2}:
 91.2857  92.1739  93.5714  91.1905  …  87.0  88.0  -1.0  -1.0  -1.0  85.0
 91.3125  94.4286  -1.0     91.3571     -1.0  -1.0  -1.0  -1.0  90.0  -1.0
 89.8333  -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0     …  -1.0  -1.0  -1.0  82.5  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 -1.0     -1.0     -1.0     -1.0     …  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 89.6     -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0
 88.3333  -1.0     -1.0     -1.0        -1.0  -1.0  -1.0  -1.0  -1.0  -1.0


# Lets find the fraction of values against missing values

In [12]:
sum(A .> -1)/length(A)

0.038902522428289815

In [257]:
threshold = 4

skip_columns = falses(n_wineries)
for col = 1:n_wineries
    n_not_missing = sum(A[:, col] .> -1)
    if n_not_missing > threshold
        skip_columns[col] = true
    end
end

skip_rows = falses(n_varieties)
for row = 1:n_varieties
    n_not_missing = sum(A[row, :] .> -1)
    if n_not_missing > threshold
        skip_rows[row] = true
    end
end

In [258]:
println(sum(skip_columns))
println(sum(skip_rows))

48
24


In [418]:
data_matrix = A[skip_rows, skip_columns]
OG = deepcopy(data_matrix)
mask = data_matrix .> 0
avg_rating = sum(data_matrix[mask])/length(data_matrix[mask])
data_matrix[mask] = data_matrix[mask] .- avg_rating
data_matrix

24×48 Array{Float64,2}:
  4.0649   -1.0       -1.0       -0.99004   …  -1.0       -5.22081   -2.97081
  4.09169   0.279191  -1.0       -1.0          -1.0       -1.0       -5.22081
  2.61252  -1.0       -1.0       -1.36367      -1.0       -4.22081   -4.22081
 -1.0      -1.0       -1.0       -1.22081      -1.0       -1.0       -3.22081
 -1.0      -1.0       -1.0       -1.0          -1.0       -1.0       -1.0    
 -1.0      -1.0       -1.0       -1.0       …  -1.0       -1.0       -1.0    
 -1.0      -0.220809   1.77919   -2.22081      -1.0        0.779191  -4.22081
 -1.0      -1.0       -1.0       -1.0          -1.0       -1.0       -4.22081
 -1.0      -1.0        0.779191   0.779191     -1.0       -1.0       -1.0    
  2.37919  -6.22081   -1.0       -1.0          -1.0       -1.0       -3.22081
  1.11252  -1.0       -1.0       -1.0       …  -1.0       -1.0       -1.0    
 -1.0      -1.0       -1.0       -1.0          -1.0       -6.22081   -1.0    
 -1.0      -1.0       -0.220809  -1.0   

In [419]:
sum(data_matrix .> -1)/length(data_matrix)

0.15625

In [431]:
function SGD(X::Array{Float64, 2}, k::Int64, γ::Float64, n_epoch::Int64)
    n_variety, n_winery = size(X)
    p = rand(n_variety, k) .* 0.2 .- 0.1
    q = rand(n_winery, k) .* 0.2 .- 0.1
    
    for ep = 1:n_epoch
        for i = 1:n_variety
            for j = 1:n_winery
                if X[i,j] < 0
                    continue
                end
                err = X[i,j] - dot(p[i, :], q[j, :])
                q_ = err * q[j, :]
                p_ = err * p[i, :]
                p[i, :] += γ * q_
                q[j, :] += γ * p_
            end
        end
    end
    return p, q
end

SGD (generic function with 1 method)

In [432]:
p, q = SGD(data_matrix, 10, 0.01, 100)

([0.816621 -0.418353 … 0.073985 0.593846; 0.255447 0.10275 … 0.0179691 0.328851; … ; 1.34412 -0.461248 … 0.894955 -0.28936; 1.32862 -0.646737 … 0.836486 -0.400612], [0.466949 -0.0867776 … -0.232825 -0.00341132; 0.14248 -0.0242611 … -0.0206372 0.0377417; … ; 0.126504 -0.119769 … 0.0133615 0.0662394; 0.0281446 -0.02607 … -0.00785101 0.0707364])

In [433]:
function estimate(wine, variety)
    @printf("Winery: %s\nVariety: %s\n", wineries[wine], varieties[variety])
    return dot(p[wine], q[variety]) + avg_rating
end

estimate (generic function with 1 method)

In [434]:
estimate(5, 3)

Winery: Panther Creek
Variety: Pinot Gris


87.12024249848974

In [436]:
new_ratings = p*q'
rmse = sqrt(sum((data_matrix[mask].-new_ratings[mask]).^2)/length(data_matrix))
@printf("Root Mean Squared Error: %f\n", rmse)

Root Mean Squared Error: 1.120706


In [360]:
function SGD_regularized(X::Array{Float64, 2}, k::Int64, γ::Float64, n_epoch::Int64, λ::Float64)
    n_variety, n_winery = size(X)
    p = rand(n_variety, k) .* 2 .- 1
    q = rand(n_winery, k) .* 2 .- 1
    
    for ep = 1:n_epoch
        for i = 1:n_variety
            for j = 1:n_winery
                if X[i,j] < 0
                    continue
                end
                err = X[i,j] - dot(p[i, :], q[j, :])
                q_ = err * q[j, :]
                p_ = err * p[i, :]
                p[i, :] += γ * (q_ - λ .* p[i, :])
                q[j, :] += γ * (p_ - λ .* q[j, :])
            end
        end
    end
    return p, q
end

SGD_regularized (generic function with 1 method)

In [427]:
p, q = SGD_regularized(data_matrix, 10, 0.01, 1000, 0.02)

([-0.246461 0.617844 … -0.432837 -0.115429; -0.0289538 -0.178223 … -0.476018 -0.305676; … ; -1.13563 0.624995 … -0.844984 -0.304396; -0.883505 1.84268 … -0.354454 -0.666433], [-0.532737 -0.222233 … -0.647947 -0.504887; -0.179209 0.560603 … 0.414415 -0.217158; … ; 0.558224 0.256096 … 0.235618 0.266347; -0.880881 0.403641 … 0.730178 -0.611976])

In [437]:
new_ratings_regularized = p*q'
rmse_regularized = sqrt(sum((data_matrix[mask].-new_ratings_regularized[mask]).^2)/length(data_matrix))
@printf("Root Mean Squared Error for regularized model: %f\n", rmse_regularized)

Root Mean Squared Error for regularized model: 1.120706


In [429]:
new_ratings_regularized .+ average_rating

24×48 Array{Float64,2}:
 91.3913  87.0707  89.2168  86.8622  …  88.4693  86.9373  89.3271  88.6013
 91.3178  87.5966  87.5279  86.4009     89.0724  86.6607  88.0337  88.4641
 90.0644  87.7838  88.8643  85.6786     88.1227  87.6614  88.1705  89.7892
 86.2496  87.6029  87.2003  88.241      87.7114  89.1274  88.2716  85.6729
 88.087   87.3035  88.7185  87.1119     85.7461  87.0139  88.1814  87.8346
 89.31    87.0693  88.2289  87.0145  …  88.3553  86.8764  88.2332  87.2754
 89.1148  86.3302  89.2465  88.4918     86.6484  84.7548  88.2474  87.6477
 88.1893  86.5168  88.8003  87.3084     87.8197  84.8569  87.2536  88.7284
 88.3528  86.6167  88.0275  88.0205     86.9521  85.655   87.6899  87.5284
 90.1268  89.0414  88.4873  86.8238     89.3019  88.3309  89.4775  88.8896
 88.2417  86.5928  87.9251  87.113   …  86.9167  86.5113  87.4308  87.6678
 88.2384  87.5754  88.7023  86.6849     87.2517  87.92    88.2487  87.2741
 88.2151  86.6199  88.2916  87.8787     87.0143  86.1644  87.926   86.0008
 

In [430]:
new_ratings .+ average_rating

24×48 Array{Float64,2}:
 91.5542  87.7967  90.7919  88.0734  …  87.3681  87.3663  87.6469  87.4399
 91.0967  87.6065  89.8539  87.5971     87.2649  87.443   87.2288  87.5075
 90.3383  87.6112  89.2179  88.008      87.4319  87.5074  87.6203  87.6317
 87.5165  87.3701  87.3594  87.41       87.3865  87.3834  87.3842  87.3887
 87.7595  87.3748  87.7396  87.4331     87.3562  87.3657  87.4389  87.3771
 89.2444  87.5476  88.4756  87.6983  …  87.3941  87.525   87.4791  87.5394
 89.5327  87.7524  89.2428  88.6675     87.4726  87.4926  88.2513  87.5384
 90.0703  87.8217  90.1801  87.4714     87.4026  87.5431  87.3511  87.5246
 88.0794  87.4326  88.0706  87.8788     87.2834  87.257   87.6413  87.2359
 90.2513  88.0934  89.7523  88.1077     87.6415  87.9734  87.8807  87.7955
 88.2559  87.2415  88.0355  87.695   …  87.136   87.1222  87.4601  87.117 
 88.8625  87.4242  89.211   87.937      87.1656  87.1991  87.6649  87.195 
 89.0748  87.5885  88.7587  87.8583     87.3914  87.4131  87.668   87.496 
 